In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import torch
from scipy import stats
from torch.nn.utils.rnn import pad_sequence
from sklearn.neural_network import MLPRegressor
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem
from sklearn.metrics import mean_squared_error, r2_score
import time
import os
from tqdm import tqdm
from scipy.stats import shapiro
from scipy.stats import normaltest
from scipy.stats import ttest_ind
from scipy.stats import mannwhitneyu
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
def create_statistics(vals):
    values = vals.copy()
    mols = [Chem.MolFromSmiles(x) for x in values[SMILES_COLUMN]]
    
    atom_count = [mol.GetNumAtoms() for mol in mols]
    values['atom_count'] = pd.Series(atom_count)
    
    values['ring_count'] = pd.Series([mol.GetRingInfo().NumRings() for mol in mols])
    
    mol_atoms = [mol.GetAtoms() for mol in mols]
    non_simple_atom_count = [sum((num.GetAtomicNum() not in [1, 6, 8, 7]) for num in m_atoms) for m_atoms in mol_atoms]
    values['non_organogens_count'] = pd.Series(non_simple_atom_count)
    
    values['non_organogens_percent'] = values.non_organogens_count / values.atom_count *100
    
    benzene_smiles = 'C1=CC=CC=C1'
    pattern = Chem.MolFromSmiles(benzene_smiles)
    values['aromatic_count'] = pd.Series([len(mol.GetSubstructMatches(pattern)) for mol in mols])
    
    values['aromatic_rings_percent'] = values.aromatic_count / values.ring_count *100
    values['aromatic_rings_percent'] = values['aromatic_rings_percent'].fillna(0)
    
    values['bond_count'] =  [len(mol.GetBonds()) for mol in mols]
    values['aromatic_bond_count'] = [sum(bond.GetIsAromatic() for bond in mol.GetBonds()) for mol in mols]
    values['aromatic_bond_percent'] = values.aromatic_bond_count / values.bond_count * 100
    
    values_list = [pd.Series(Chem.CanonicalRankAtoms(mol, breakTies=False)).value_counts() for mol in mols]
    values['is_symmetric'] =  [(len(values) - 1 <= len(values[values % 2 == 0])) or (len(values) - 1 <= len(values[(values % 2 == 1) & (values > 1)])) for values in values_list]
    
    return values

In [3]:

FILENAME = "logp_wo_logp_json_wo_averaging_test_predictions"

SMILES_COLUMN = 'smiles'
ERROR_COLUMN = 'Squared Error'

VALUE_COLUMNS = ['logP']

FOLDS= 5

N_VALUES = 200

In [4]:
kruskal_stistics = []
kruskal_stats = {}
kruskal_stats_mean_std = {}

DATA_PATH = "../../../data/raw/baselines/morganfp"
rmse = []

preds = []

for FOLD in range(FOLDS):
    import json
    PRED_COLUMNS = [value+'_pred' for value in VALUE_COLUMNS]
    kruskal_stats_mean_std['Model'] = 'Morgan FP'
    kruskal_stats_mean_std['Data'] = ' '.join(VALUE_COLUMNS)

    for VALUE_COLUMN in VALUE_COLUMNS:
        if VALUE_COLUMN in kruskal_stats.keys():
            pass
        else:
            kruskal_stats[VALUE_COLUMN] = {}
            kruskal_stats[VALUE_COLUMN]['symmetric test RMSE'] = []
            kruskal_stats[VALUE_COLUMN]['symmetric test R2'] = []
            kruskal_stats[VALUE_COLUMN]['non-symmetric test RMSE'] = []
            kruskal_stats[VALUE_COLUMN]['non-symmetric test R2'] = []
            kruskal_stats[VALUE_COLUMN]['rmse']= []
            kruskal_stats[VALUE_COLUMN]['r2'] = []
        PRED_COLUMN = VALUE_COLUMN+'_pred'
        test_prediction = pd.read_csv(os.path.join(DATA_PATH,FILENAME+'_'+str(FOLD)+'.csv'))
        kruskal_stats[VALUE_COLUMN]['rmse'].append(\
                                                   mean_squared_error(test_prediction[~test_prediction[VALUE_COLUMN].isna()][VALUE_COLUMN],\
                                                                      test_prediction[~test_prediction[VALUE_COLUMN].isna()][PRED_COLUMN])**0.5)
        kruskal_stats[VALUE_COLUMN]['r2'].append(r2_score(\
                                                          test_prediction[~test_prediction[VALUE_COLUMN].isna()][VALUE_COLUMN],\
                                                          test_prediction[~test_prediction[VALUE_COLUMN].isna()][PRED_COLUMN]))
        rmse.append(mean_squared_error(test_prediction[~test_prediction[VALUE_COLUMN].isna()][VALUE_COLUMN],\
                                       test_prediction[~test_prediction[VALUE_COLUMN].isna()][PRED_COLUMN])**0.5)


        test_prediction[ERROR_COLUMN] = (test_prediction[VALUE_COLUMN] - test_prediction[PRED_COLUMN]) ** 2    
        test_prediction_sorted = test_prediction.sort_values(by=[ERROR_COLUMN], ascending=False)    

        compare_data_with_stats = create_statistics(test_prediction_sorted)

        best_n_vals = compare_data_with_stats.iloc[-N_VALUES:]
        worst_n_vals = compare_data_with_stats.iloc[:N_VALUES]


        for stat in compare_data_with_stats.columns:
            if stat not in [SMILES_COLUMN, ERROR_COLUMN] and stat not in VALUE_COLUMNS and stat not in PRED_COLUMNS:
                if stat not in kruskal_stats[VALUE_COLUMN].keys():
                    kruskal_stats[VALUE_COLUMN][stat] = []
                kruskal_stats[VALUE_COLUMN][stat].append(stats.kruskal(best_n_vals[stat], worst_n_vals[stat])[1])

        symmetric_rull = lambda values: (len(values) - 1 <= len(values[values % 2 == 0])) or\
        (len(values) - 1 <= len(values[(values % 2 == 1) & (values > 1)]))
        values_list = [pd.Series(Chem.CanonicalRankAtoms(Chem.MolFromSmiles(smiles), breakTies=False)).value_counts()\
                       for smiles in\
                       compare_data_with_stats[~compare_data_with_stats[VALUE_COLUMN].isna()][SMILES_COLUMN]]
        symmetric_indices = [symmetric_rull(values) for values in values_list]
        not_symmetric_indices = [not e for e in symmetric_indices]

        symmetric_y_predicted = compare_data_with_stats[~compare_data_with_stats[VALUE_COLUMN].isna()][PRED_COLUMN][symmetric_indices]
        symmetric_y_expected = compare_data_with_stats[~compare_data_with_stats[VALUE_COLUMN].isna()][VALUE_COLUMN][symmetric_indices]    
        not_symmetric_y_predicted = compare_data_with_stats[~compare_data_with_stats[VALUE_COLUMN].isna()][PRED_COLUMN][not_symmetric_indices]    
        not_symmetric_y_expected = compare_data_with_stats[~compare_data_with_stats[VALUE_COLUMN].isna()][VALUE_COLUMN][not_symmetric_indices]   

        from sklearn.metrics import mean_squared_error    

        kruskal_stats[VALUE_COLUMN]['symmetric test RMSE'].append(mean_squared_error(symmetric_y_expected, symmetric_y_predicted, squared=False))
        kruskal_stats[VALUE_COLUMN]['symmetric test R2'].append(r2_score(symmetric_y_expected, symmetric_y_predicted))


        kruskal_stats[VALUE_COLUMN]['non-symmetric test RMSE'].append(mean_squared_error(not_symmetric_y_expected, not_symmetric_y_predicted, squared=False))
        kruskal_stats[VALUE_COLUMN]['non-symmetric test R2'].append(r2_score(not_symmetric_y_expected, not_symmetric_y_predicted))


for VALUE in kruskal_stats.keys():
    for stat in kruskal_stats[VALUE].keys():
        kruskal_stats_mean_std[VALUE+'_'+stat+'_mean'] = np.mean(kruskal_stats[VALUE][stat])
        kruskal_stats_mean_std[VALUE+'_'+stat+'_std'] = np.std(kruskal_stats[VALUE][stat])

kruskal_stistics.append(kruskal_stats_mean_std)
    
kruskal_stistics = pd.DataFrame(kruskal_stistics)

import math
for i in range(len(kruskal_stistics)):
    stats_model = kruskal_stistics.iloc[i]
    print(stats_model['Model'])
    print(stats_model['Data'])
    print()
    output = {}
    for stat in stats_model.keys():
        if stat=='Model' or stat=='Data' or stat=='Dataset' or stat=='Num_exp':
            continue
        stat_name = '_'.join(stat.split('_')[:-1])
        stat_suf = stat.split('_')[-1]
        if stat_name not in output.keys():
            output[stat_name] = {}
        output[stat_name][stat_suf] = stats_model[stat]
    for stat in output.keys():
        if math.isnan(output[stat]['mean']):
            continue
        print(stat,' = ', round(output[stat]['mean'],3), '+/-', round(output[stat]['std'], 3))
    print()
    print('===')
    print()
    
values = {}
for column in list(kruskal_stistics):
    if column not in ['Model','Num_exp','Data','Dataset']:
        property_name = ' '.join(column.split('_')[:-1])
        value_name = column.split('_')[-1]
        value = kruskal_stistics[column].values
        if property_name not in values.keys():
            values[property_name] = {}
            values[property_name][value_name] = value
        else:
            values[property_name][value_name] = value
            values[property_name] = [str(round(mean,3))+'+/-'+str(round(std, 3)) for mean, std in zip(values[property_name]['mean'], values[property_name]['std'])]
stats_markdown_table = pd.DataFrame(columns = ['Model','Data'])
for column in list(stats_markdown_table):
    stats_markdown_table[column] = kruskal_stistics[column]
for column in values.keys():
    stats_markdown_table[column] = values[column]
stats_markdown_table = stats_markdown_table[['Model',
 'Data',
 'logP symmetric test RMSE',
 'logP non-symmetric test RMSE',
 'logP symmetric test R2',
 'logP non-symmetric test R2',
 'logP rmse',
 'logP r2']]
RAW_PATH = '../../../data/raw/baselines/morganfp/'
with open(os.path.join(RAW_PATH, FILENAME+'_worst_best_stats.txt'), 'w') as f:
    f.write(stats_markdown_table.to_markdown())

Morgan FP
logP

logP_symmetric test RMSE  =  0.884 +/- 0.307
logP_symmetric test R2  =  0.835 +/- 0.135
logP_non-symmetric test RMSE  =  0.543 +/- 0.02
logP_non-symmetric test R2  =  0.913 +/- 0.005
logP_rmse  =  0.565 +/- 0.042
logP_r2  =  0.908 +/- 0.013
logP_Unnamed: 0  =  0.426 +/- 0.288
logP_atom_count  =  0.413 +/- 0.269
logP_ring_count  =  0.141 +/- 0.103
logP_non_organogens_count  =  0.492 +/- 0.344
logP_non_organogens_percent  =  0.512 +/- 0.315
logP_aromatic_count  =  0.5 +/- 0.287
logP_aromatic_rings_percent  =  0.448 +/- 0.255
logP_bond_count  =  0.0 +/- 0.0
logP_aromatic_bond_count  =  0.721 +/- 0.159
logP_aromatic_bond_percent  =  0.001 +/- 0.002
logP_is_symmetric  =  0.04 +/- 0.062

===



In [5]:

FILENAME = "logd_Lip_wo_averaging_test_predictions"

SMILES_COLUMN = 'smiles'
ERROR_COLUMN = 'Squared Error'

VALUE_COLUMNS = ['logD']

FOLDS= 5

N_VALUES = 200

In [9]:
kruskal_stistics = []
kruskal_stats = {}
kruskal_stats_mean_std = {}

DATA_PATH = "../../../data/raw/baselines/morganfp"
rmse = []

preds = []

for FOLD in range(FOLDS):
    import json
    PRED_COLUMNS = [value+'_pred' for value in VALUE_COLUMNS]
    kruskal_stats_mean_std['Model'] = 'Morgan FP'
    kruskal_stats_mean_std['Data'] = ' '.join(VALUE_COLUMNS)

    for VALUE_COLUMN in VALUE_COLUMNS:
        if VALUE_COLUMN in kruskal_stats.keys():
            pass
        else:
            kruskal_stats[VALUE_COLUMN] = {}
            kruskal_stats[VALUE_COLUMN]['symmetric test RMSE'] = []
            kruskal_stats[VALUE_COLUMN]['symmetric test R2'] = []
            kruskal_stats[VALUE_COLUMN]['non-symmetric test RMSE'] = []
            kruskal_stats[VALUE_COLUMN]['non-symmetric test R2'] = []
            kruskal_stats[VALUE_COLUMN]['rmse']= []
            kruskal_stats[VALUE_COLUMN]['r2'] = []
        PRED_COLUMN = VALUE_COLUMN+'_pred'
        test_prediction = pd.read_csv(os.path.join(DATA_PATH,FILENAME+'_'+str(FOLD)+'.csv'))
        kruskal_stats[VALUE_COLUMN]['rmse'].append(\
                                                   mean_squared_error(test_prediction[~test_prediction[VALUE_COLUMN].isna()][VALUE_COLUMN],\
                                                                      test_prediction[~test_prediction[VALUE_COLUMN].isna()][PRED_COLUMN])**0.5)
        kruskal_stats[VALUE_COLUMN]['r2'].append(r2_score(\
                                                          test_prediction[~test_prediction[VALUE_COLUMN].isna()][VALUE_COLUMN],\
                                                          test_prediction[~test_prediction[VALUE_COLUMN].isna()][PRED_COLUMN]))
        rmse.append(mean_squared_error(test_prediction[~test_prediction[VALUE_COLUMN].isna()][VALUE_COLUMN],\
                                       test_prediction[~test_prediction[VALUE_COLUMN].isna()][PRED_COLUMN])**0.5)


        test_prediction[ERROR_COLUMN] = (test_prediction[VALUE_COLUMN] - test_prediction[PRED_COLUMN]) ** 2    
        test_prediction_sorted = test_prediction.sort_values(by=[ERROR_COLUMN], ascending=False)    

        compare_data_with_stats = create_statistics(test_prediction_sorted)

        best_n_vals = compare_data_with_stats.iloc[-N_VALUES:]
        worst_n_vals = compare_data_with_stats.iloc[:N_VALUES]


        for stat in compare_data_with_stats.columns:
            if stat not in [SMILES_COLUMN, ERROR_COLUMN] and stat not in VALUE_COLUMNS and stat not in PRED_COLUMNS:
                if stat not in kruskal_stats[VALUE_COLUMN].keys():
                    kruskal_stats[VALUE_COLUMN][stat] = []
                try:
                    kruskal_stats[VALUE_COLUMN][stat].append(stats.kruskal(best_n_vals[stat], \
                                                                           worst_n_vals[stat])[1])
                except ValueError:
                    kruskal_stats[VALUE_COLUMN][stat].append(np.nan) 

        symmetric_rull = lambda values: (len(values) - 1 <= len(values[values % 2 == 0])) or\
        (len(values) - 1 <= len(values[(values % 2 == 1) & (values > 1)]))
        values_list = [pd.Series(Chem.CanonicalRankAtoms(Chem.MolFromSmiles(smiles), breakTies=False)).value_counts()\
                       for smiles in\
                       compare_data_with_stats[~compare_data_with_stats[VALUE_COLUMN].isna()][SMILES_COLUMN]]
        symmetric_indices = [symmetric_rull(values) for values in values_list]
        not_symmetric_indices = [not e for e in symmetric_indices]

        symmetric_y_predicted = compare_data_with_stats[~compare_data_with_stats[VALUE_COLUMN].isna()][PRED_COLUMN][symmetric_indices]
        symmetric_y_expected = compare_data_with_stats[~compare_data_with_stats[VALUE_COLUMN].isna()][VALUE_COLUMN][symmetric_indices]    
        not_symmetric_y_predicted = compare_data_with_stats[~compare_data_with_stats[VALUE_COLUMN].isna()][PRED_COLUMN][not_symmetric_indices]    
        not_symmetric_y_expected = compare_data_with_stats[~compare_data_with_stats[VALUE_COLUMN].isna()][VALUE_COLUMN][not_symmetric_indices]   

        from sklearn.metrics import mean_squared_error    

        kruskal_stats[VALUE_COLUMN]['symmetric test RMSE'].append(mean_squared_error(symmetric_y_expected, symmetric_y_predicted, squared=False))
        kruskal_stats[VALUE_COLUMN]['symmetric test R2'].append(r2_score(symmetric_y_expected, symmetric_y_predicted))


        kruskal_stats[VALUE_COLUMN]['non-symmetric test RMSE'].append(mean_squared_error(not_symmetric_y_expected, not_symmetric_y_predicted, squared=False))
        kruskal_stats[VALUE_COLUMN]['non-symmetric test R2'].append(r2_score(not_symmetric_y_expected, not_symmetric_y_predicted))


for VALUE in kruskal_stats.keys():
    for stat in kruskal_stats[VALUE].keys():
        kruskal_stats_mean_std[VALUE+'_'+stat+'_mean'] = np.nanmean(kruskal_stats[VALUE][stat])
        kruskal_stats_mean_std[VALUE+'_'+stat+'_std'] = np.nanstd(kruskal_stats[VALUE][stat])

kruskal_stistics.append(kruskal_stats_mean_std)
    
kruskal_stistics = pd.DataFrame(kruskal_stistics)

import math
for i in range(len(kruskal_stistics)):
    stats_model = kruskal_stistics.iloc[i]
    print(stats_model['Model'])
    print(stats_model['Data'])
    print()
    output = {}
    for stat in stats_model.keys():
        if stat=='Model' or stat=='Data' or stat=='Dataset' or stat=='Num_exp':
            continue
        stat_name = '_'.join(stat.split('_')[:-1])
        stat_suf = stat.split('_')[-1]
        if stat_name not in output.keys():
            output[stat_name] = {}
        output[stat_name][stat_suf] = stats_model[stat]
    for stat in output.keys():
        if math.isnan(output[stat]['mean']):
            continue
        print(stat,' = ', round(output[stat]['mean'],3), '+/-', round(output[stat]['std'], 3))
    print()
    print('===')
    print()
    
values = {}
for column in list(kruskal_stistics):
    if column not in ['Model','Num_exp','Data','Dataset']:
        property_name = ' '.join(column.split('_')[:-1])
        value_name = column.split('_')[-1]
        value = kruskal_stistics[column].values
        if property_name not in values.keys():
            values[property_name] = {}
            values[property_name][value_name] = value
        else:
            values[property_name][value_name] = value
            values[property_name] = [str(round(mean,3))+'+/-'+str(round(std, 3)) for mean, std in zip(values[property_name]['mean'], values[property_name]['std'])]
stats_markdown_table = pd.DataFrame(columns = ['Model','Data'])
for column in list(stats_markdown_table):
    stats_markdown_table[column] = kruskal_stistics[column]
for column in values.keys():
    stats_markdown_table[column] = values[column]
stats_markdown_table = stats_markdown_table[['Model',
 'Data',
 'logD symmetric test RMSE',
 'logD non-symmetric test RMSE',
 'logD symmetric test R2',
 'logD non-symmetric test R2',
 'logD rmse',
 'logD r2']]
RAW_PATH = '../../../data/raw/baselines/morganfp/'
with open(os.path.join(RAW_PATH, FILENAME+'_worst_best_stats.txt'), 'w') as f:
    f.write(stats_markdown_table.to_markdown())

Morgan FP
logD

logD_symmetric test RMSE  =  1.179 +/- 0.354
logD_symmetric test R2  =  0.038 +/- 0.444
logD_non-symmetric test RMSE  =  0.743 +/- 0.032
logD_non-symmetric test R2  =  0.615 +/- 0.015
logD_rmse  =  0.749 +/- 0.03
logD_r2  =  0.61 +/- 0.015
logD_Unnamed: 0  =  0.451 +/- 0.343
logD_atom_count  =  0.64 +/- 0.192
logD_ring_count  =  0.392 +/- 0.168
logD_non_organogens_count  =  0.567 +/- 0.179
logD_non_organogens_percent  =  0.518 +/- 0.135
logD_aromatic_count  =  0.54 +/- 0.127
logD_aromatic_rings_percent  =  0.432 +/- 0.244
logD_bond_count  =  0.049 +/- 0.067
logD_aromatic_bond_count  =  0.056 +/- 0.101
logD_aromatic_bond_percent  =  0.471 +/- 0.325
logD_is_symmetric  =  0.169 +/- 0.228

===

